In [3]:
import random
import numpy as np
import pandas as pd

from snpp.utils.data import load_train_test_graphs
from snpp.cores.lowrank import alq_spark, predict_signs

dataset = 'slashdot'
random_seed = 123456

recache_input = False

random.seed(random_seed)
np.random.seed(random_seed)

In [23]:
from snpp.utils.signed_graph import fill_diagonal
train_g, test_g = load_train_test_graphs(dataset, recache_input)

train_m = nx.to_scipy_sparse_matrix(train_g, weight='sign', format='csr')
train_m = fill_diagonal(train_m)  # forget-me-not
truth = set((i, j, test_g[i][j]['sign']) for i, j in test_g.edges_iter())


loading train and test graphs...


100%|██████████| 77357/77357 [00:01<00:00, 59768.52it/s]


In [25]:
print(train_g.number_of_nodes())
print(test_g.number_of_nodes())
print(train_m.shape)
print(train_m[0, 0])

77357
77357
(77357, 77357)
1.0


In [15]:
param_grid = {
    'lambda_': [0.1, 0.5],
    'k': [10],
    'iterations': [20]
}


In [31]:
from itertools import product
from copy import copy
from tqdm import tqdm

rows = []
for values in tqdm(list(product(*param_grid.values()))):
    param_instance = dict(zip(param_grid.keys(), values))
    print(param_instance)
    X, Y = alq_spark(train_m, sc=sc,                 
                     seed=random_seed,
                     **param_instance)
    preds = predict_signs(X, Y, test_g.edges(), sc)
    row = copy(param_instance)
    row['accuracy'] = len(truth.intersection(preds)) / len(truth)
    print(row['accuracy'])
    rows.append(row)
df = pd.DataFrame.from_records(rows)

  0%|          | 0/2 [00:00<?, ?it/s]

{'k': 10, 'lambda_': 0.1, 'iterations': 20}


 50%|█████     | 1/2 [00:46<00:46, 46.11s/it]

0.8545820589260134
{'k': 10, 'lambda_': 0.5, 'iterations': 20}


100%|██████████| 2/2 [01:27<00:00, 44.59s/it]

0.832146037399822


In [32]:
from IPython.display import display
display(df)

,accuracy,iterations,k,lambda_
0,0.854582,20,10,0.1
1,0.832146,20,10,0.5
